In [7]:
from environment import Environment, FixedBudgetEnvironment
import matplotlib.pyplot as plt
import math
import numpy as np
from game import SumUnanimityGames, GlobalFeatureImportance, LocalFeatureImportance, UnsupervisedFeatureImportance
from plot import plot
import util

In [8]:
from algorithms.CMCS import *
from algorithms.GapE import GapE
from algorithms.SAR import SAR
from algorithms.ApproShapley import ApproShapley
from algorithms.BUS import BUS
from algorithms.HybridApproBUS import SmartHybridApproBUS
from algorithms.SVARM import SVARM, StratSVARM
from algorithms.ShapleySort import ShapleySort
from algorithms.KernelSHAP import KernelSHAP
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
n=16
k=8
budget=1000
rounds=3000
step_interval=50
metric = "ratio"
env = Environment(n=n, budget=budget, metric=metric)
results = []

In [10]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

In [11]:
filepath_global = "datasets/Global feature importance/Bank marketing classification random forest.csv"
filepath_unsupervised = "datasets/Unsupervised Feature Importance/vf_BigFive.csv"
directory_local = "datasets/image classifier"
game = GlobalFeatureImportance(filepath=filepath_global, num_players=n, use_cached=True)
# game = LocalFeatureImportance(directory=directory_local, num_players=n)
# game = UnsupervisedFeatureImportance(filepath=filepath_unsupervised, num_players=n)
# game = SumUnanimityGames()

[0.00 -0.01 0.00 ... 0.01 0.01 0.01]
[-0.00 -0.00 0.00 0.00 -0.00 -0.01 -0.00 0.00 0.00 0.00 0.00 0.00 -0.00
 0.00 0.00 0.01] 0.008106116433308063


In [ ]:
values = game.values[np.newaxis, :]
num_games = 1
shapley_values = game.phi[np.newaxis, :]

In [ ]:
var = util.calc_variance(n, values, shapley_values)
cov = util.calc_covariance(n, values, shapley_values)

In [12]:
# results += [("KernelSHAP", env.evaluate(game, KernelSHAP(), k, step_interval, rounds))]

In [13]:
results += [("Variance_CMCS_Cheat", env.evaluate(game, Variance_CMCS_Cheat(diff_variance=var), k, step_interval, rounds))]

NameError: name 'var' is not defined

In [ ]:
# results += [("Variance_CMCS", env.evaluate(game, Variance_CMCS(), k, step_interval, rounds))]

In [ ]:
results += [("Selective_CMCS", env.evaluate(game, Selective_CMCS(), k, step_interval, rounds))]

c:\Users\Tim\Documents\LMU - HiWi\top-k-shapley\algorithms\CMCS.py:259: RuntimeWarning: invalid value encountered in divide
  weights = (max_certainty - certainty) / (max_certainty - min_certainty)


In [ ]:
results += [("CMCS", env.evaluate(game, CMCS(), k, step_interval, rounds))]

In [ ]:
results += [("CMCS_Independent", env.evaluate(game, CMCS_Independent(), k, step_interval, rounds))]

In [ ]:
results += [("CMCS_Dependent", env.evaluate(game, CMCS_Dependent(), k, step_interval, rounds))]

In [ ]:
results += [("CMCS_Length", env.evaluate(game, CMCS_Length(), k, step_interval, rounds))]

In [ ]:
# results += [("StratSVARM(start_exact=True, theoretical_distribution=False)", env.evaluate(game, StratSVARM(start_exact=True, theoretical_distribution=False), k, step_interval, rounds))]

In [ ]:
# results += [("ApproShapley(optimized)", env.evaluate(game, ApproShapley(optimize=True), k, step_interval, rounds))]

In [ ]:
matrix = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
index_array = np.array([0, 0, 1])

# Use advanced indexing
result = matrix[np.arange(len(index_array)), index_array]
print(result)

[1 4 8]


In [ ]:
start_at = 0
plot(results, start_at_x=start_at, step_interval=step_interval, types=["topk", "mse"], metric=metric, save=True, filepath=f"results/{game.__class__.__name__}({game.name})_n={n}_k={k}_T={budget}_r={rounds}_{metric}_start_at={start_at}.pdf")

In [ ]:
# def pdf(x, mu, variance):
#     return  np.exp((-(x-mu)**2)/(2*variance)) / np.sqrt(2*math.pi*variance)

In [ ]:
# plt.style.use(plt.style.library['ggplot'])
# plt.rcParams.update({'font.size': 12})
# plt.rcParams['legend.frameon'] = False
# plt.rcParams['lines.markersize'] = 10
# fig, ax = plt.subplots(1, 1, figsize=(8,5), dpi=1000)
# x = np.arange(100)/10 - 5
# target_mu = 3
# target_sigma = 1
# values1 = pdf(x, 0, 1)
# values2 = pdf(x, 1, 1)
# samples = np.random.normal(1, 1, (10000))
# weights = pdf(samples, 0, 1) / pdf(samples, 1, 1)
# sum_weights = np.sum(weights)
# new_samples = np.zeros((1000))
# for j in range(1000):
#     totalweight = 0
#     for i, sample in enumerate(samples):
#         totalweight += weights[i]
#         if np.random.rand() < weights[i]/totalweight:
#             new_samples[j] = sample
#     # new_samples[j] = np.random.choice(samples, p=weights/sum_weights)

# def histo(samples):
#     num, edges = np.histogram(samples, bins=100)
#     x = edges[:-1]
#     width = edges[1:]-edges[:-1]
#     area = np.sum(num * width)
#     densities = num/area
#     return x, densities, width 
# ax.plot(x, values1, ".-", linewidth=2.0)
# ax.plot(x, values2, ".-", linewidth=2.0)
# preX, preDensity, preWidth  = histo(samples)
# newX, newDensity, newWidth  = histo(new_samples)
# ax.bar(preX, preDensity, preWidth, alpha=0.8)
# ax.bar(newX, newDensity, newWidth, alpha=0.8)
# # ax.plot(np.sort(samples), pdf(np.sort(samples), 1, 1), ".-", linewidth=2.0)